# PSII Tutorial 

In [ ]:
import sys, traceback
import cv2
import numpy as np
import argparse
import string
from plantcv import plantcv as pcv

In [ ]:
class options:
    def __init__(self):
        self.image = "img/tutorial_images/psII/Fmax.jpg"
        self.debug = "plot"
        self.writeimg= False 
        self.result = "./psII_tutorial_results"
        self.outdir = "."
# Get options
args = options()

# Set debug to the global parameter 
pcv.params.debug = args.debug

In [ ]:
# Read image (converting fmax and track to 8 bit just to create a mask, use 16-bit for all the math)
mask, path, filename = pcv.readimage(args.image)

# Read in track region of interest 
track = cv2.imread("img/tutorial_images/psII/mask.jpg")

# Plot the track mask out manually since we aren't using a PlantCV function
pcv.plot_image(track)

# Make a copy of our image 
mask1 = mask

We use a premade mask for the screw of the car that consistently give background signal, 
but this is not required. 

In [ ]:
# Mask pesky track autofluor

# Inputs:
#   rgb_img - RGB image data 
# channel - Split 'h' (hue), 's' (saturation), or 'v' (value) channel 
track1 = pcv.rgb2gray_hsv(track, 'v')

# Inputs:
#   gray_img - Grayscale image data 
#   threshold- Threshold value (between 0-255)
#   max_value - Value to apply above threshold (255 = white) 
#   object_type - 'light' (default) or 'dark'. If the object is lighter than the background then standard threshold is done.
#                 If the object is darker than the background then inverse thresholding is done. 
track_thresh = pcv.threshold.binary(track1, 0, 255, 'light')

# Inputs:
#   gray_img - Grayscale image data 
track_inv = pcv.invert(track_thresh)

# Inputs: 
#   rgb_img - RGB image data 
#   mask - Binary mask image data 
#   mask_color - 'black' or 'white'
track_masked = pcv.apply_mask(mask1, track_inv, 'black')

In [ ]:
# Threshold the image
fmax_thresh = pcv.threshold.binary(track_masked, 20, 255, 'light')

In [ ]:
# Median Filter

# Inputs:
#   gray_img - Grayscale image data 
#   ksize - Kernel size. Integer or tuple; (ksize, ksize) box if integer is input, 
#           (n, m) size box if tuple is given.
s_mblur = pcv.median_blur(fmax_thresh, 5)
s_cnt = pcv.median_blur(fmax_thresh, 5)

In [ ]:
# Fill small objects

# Inputs:
#   bin_img - Binary image data 
#   size - Minimum object area size in pixels (integer), smaller objects get filled in. 
s_fill = pcv.fill(s_mblur, 110)
sfill_cnt = pcv.fill(s_mblur, 110)

In [ ]:
# Identify objects

# Inputs:
#   img - RGB or grayscale image data for plotting
#   mask - Binary mask used for detecting contours
id_objects,obj_hierarchy = pcv.find_objects(mask, sfill_cnt)

In [ ]:
# Define ROI

# Inputs: 
#   x - The x-coordinate of the upper left corner of the rectangle 
#   y - The y-coordinate of the upper left corner of the rectangle 
#   h - The height of the rectangle 
#   w - The width of the rectangle 
#   img - RGB or grayscale image to plot the ROI on 
roi1, roi_hierarchy = pcv.roi.rectangle(x=100, y=100, h=200, w=200, img=mask)

In [ ]:
# Decide which objects to keep

# Inputs:
#   img - RGB or grayscale image data to display kept objects on 
#   roi_type - 'cutto' or 'partial' => include objexts that are partially inside or overlapping with the ROI 
#   roi_contour - contour of ROI, output from pcv.roi.rectangle in this case
#   object_contour - contour of objects, output from pcv.roi.rectangle in this case 
#   obj_hierarchy - heirarch of objects, output from pcv.find_objects function 
roi_objects, hierarchy3, kept_mask, obj_area = pcv.roi_objects(mask, 'partial', roi1, roi_hierarchy, id_objects, obj_hierarchy)

In [ ]:
# Object combine kept objects

# Use the object_composition function to outline the plant 
# Inputs:
#   img - RGB or grayscale image data for plotting 
#   contours - Contour list 
#   hierarchy - Contour hierarchy array 
obj, masked = pcv.object_composition(mask, roi_objects, hierarchy3)

Now we can analyze the plant objects for traits such as shape, or PSII signal. 

In [ ]:
################ Analysis ################  

# Find shape properties, output shape image (optional)
shape_header, shape_data, shape_img = pcv.analyze_object(mask, obj, masked, args.outdir + '/' + filename)

# Fluorescence Measurement (read in 16-bit images)
fdark = cv2.imread("img/tutorial_images/psII/Fdark.jpg", -1)
fmin = cv2.imread("img/tutorial_images/psII/Fmin.jpg", -1)
fmax = cv2.imread("img/tutorial_images/psII/Fmax.jpg", -1)

In [ ]:
# Extract Fv/Fm data of objects and produce pseudocolored images 

# Inputs:
#   fdark - Grayscale image 
#   fmin - Grayscale image 
#   fmax - Grayscale image 
#   mask - Binary mask of selected contours 
#   filename - False (default) or image name. If defined, print image.
#   bins - Number of grayscale bins (0-256 for 8-bit img, 0-65536 for 16-bit). Default bins = 256
fvfm_header, fvfm_data, fvfm_images = pcv.fluor_fvfm(fdark,fmin,fmax,kept_mask, args.outdir+'/'+filename, 1000)

In [ ]:
# Write shape and nir data to results file
result=open(args.result,"a")
result.write('\t'.join(map(str,shape_header)))
result.write("\n")
result.write('\t'.join(map(str,shape_data)))
result.write("\n")
for row in shape_img:  
    result.write('\t'.join(map(str,row)))
    result.write("\n")
result.write('\t'.join(map(str,fvfm_header)))
result.write("\n")
result.write('\t'.join(map(str,fvfm_data)))
result.write("\n")
result.close()